In [1]:
from datetime import timedelta
from azure.core.credentials import AzureKeyCredential
from azure.search.documents.indexes import SearchIndexerClient, SearchIndexClient
from azure.search.documents.indexes.models import (
    SearchIndexerDataSourceConnection,
    SearchIndexerDataContainer,
    SearchIndexerSkillset,
    WebApiSkill,
    AzureOpenAIEmbeddingSkill,
    InputFieldMappingEntry,
    OutputFieldMappingEntry,
    SearchIndexer,
    SearchIndex,
    SearchField,
    SearchFieldDataType,
    VectorSearch,
    VectorSearchProfile,
    HnswAlgorithmConfiguration,
    AzureOpenAIVectorizer,
    AzureOpenAIVectorizerParameters,
    SemanticConfiguration,
    SemanticPrioritizedFields,
    SemanticField,
    SemanticSearch,
    IndexingParameters,
    SearchIndexerIndexProjection,
    SearchIndexerIndexProjectionsParameters,
    IndexProjectionMode,
    SearchIndexerIndexProjectionSelector,
    IndexingSchedule,
)

In [2]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

search_endpoint = os.getenv("SEARCH_ENDPOINT")
search_client_key = os.getenv("SEARCH_CLIENT_KEY")
storage_connection_string = os.getenv("STORAGE_CONNECTION_STRING")
blob_container_name = os.getenv("BLOB_CONTAINER_NAME")
azure_function_url = os.getenv("AZURE_FUNCTION_URL")
azure_openai_resource_url = os.getenv("AZURE_OPENAI_RESOURCE_URL")
azure_openai_api_key = os.getenv("AZURE_OPENAI_API_KEY")
azure_openai_embedding_model_name = os.getenv("AZURE_OPENAI_EMBEDDING_MODEL_NAME")
azure_openai_embedding_deployment_name  = os.getenv("AZURE_OPENAI_EMBEDDING_DEPLOYMENT_NAME")
azure_openai_embedding_dimensions = int(os.getenv("AZURE_OPENAI_EMBEDDING_DIMENSIONS", 3072))

In [3]:
index_name = "video-index-manual"

In [4]:
index_client = SearchIndexClient(
    endpoint=search_endpoint,
    credential=AzureKeyCredential(search_client_key)
)

### Create index

In [5]:
fields = [
    # SearchField(name="parent_id", type=SearchFieldDataType.String, filterable=True, sortable=True),
    SearchField(name="chunk_id", type=SearchFieldDataType.String, key=True, filterable=True, analyzer_name="keyword"),
    SearchField(name="id", type=SearchFieldDataType.String, searchable=True, sortable=True, filterable=True),
    SearchField(name="video_name", type=SearchFieldDataType.String, searchable=True, filterable=True),
    SearchField(name="text", type=SearchFieldDataType.String, searchable=True),
    SearchField(name="start_time", type=SearchFieldDataType.String, filterable=True, sortable=True),
    SearchField(name="end_time", type=SearchFieldDataType.String, filterable=True, sortable=True),
    SearchField(
        name="vector",
        type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
        searchable=True,
        vector_search_dimensions=azure_openai_embedding_dimensions,
        vector_search_profile_name="vector-profile"
    )
]
vector_search = VectorSearch(
    profiles=[
        VectorSearchProfile(
            name="vector-profile", 
            algorithm_configuration_name="hnsw-config",
            vectorizer_name="azure-openai-vectorizer",
        )
    ],
    algorithms=[
        HnswAlgorithmConfiguration(name="hnsw-config")
    ],
    vectorizers=[
        AzureOpenAIVectorizer(
            vectorizer_name="azure-openai-vectorizer",
            parameters=AzureOpenAIVectorizerParameters(
                resource_url=azure_openai_resource_url,
                deployment_name=azure_openai_embedding_deployment_name,
                model_name=azure_openai_embedding_model_name,
                api_key=azure_openai_api_key,
            )
        )
    ]
)

semantic_config = SemanticConfiguration(  
    name="semantic-config",  
    prioritized_fields=SemanticPrioritizedFields(  
        content_fields=[SemanticField(field_name="text")]  
    )
)

semantic_search = SemanticSearch(
    default_configuration_name="semantic-config",
    configurations=[semantic_config]
)

index = SearchIndex(name=index_name, fields=fields, vector_search=vector_search, semantic_search=semantic_search)
index_client.create_or_update_index(index)
print("✅ Index with vector field created")

✅ Index with vector field created
